# Model Context Protocol (MCP) - Hands-On Implementation Guide

> Practical, runnable examples for building MCP servers and clients with Python SDK

**Created:** January 2025  
**Version:** 1.0  
**Prerequisites:** Python 3.10+, MCP SDK installed

---

## Table of Contents

1. [Setup & Installation](#1-setup-installation)
2. [Building Your First MCP Server](#2-building-your-first-mcp-server)
3. [Implementing Tools](#3-implementing-tools)
4. [Implementing Resources](#4-implementing-resources)
5. [Implementing Prompts](#5-implementing-prompts)
6. [Building an MCP Client](#6-building-an-mcp-client)
7. [Real-World Examples](#7-real-world-examples)
8. [Testing & Debugging](#8-testing-debugging)
9. [Deployment Patterns](#9-deployment-patterns)
10. [Complete Project Examples](#10-complete-project-examples)

---

## 1. Setup & Installation

### 1.1 Install MCP SDK

In [ ]:
# Install MCP Python SDK
!pip install mcp --quiet

# Install additional dependencies
!pip install httpx pydantic python-dotenv --quiet

print("✅ MCP SDK installed successfully!")

### 1.2 Verify Installation

In [ ]:
import mcp
from mcp.server import Server
from mcp.types import Tool, Resource, Prompt, TextContent

print(f"MCP SDK Version: {mcp.__version__ if hasattr(mcp, '__version__') else 'Latest'}")
print("✅ All imports successful!")

### 1.3 Environment Setup

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Set up basic configuration
SERVER_NAME = "demo-mcp-server"
SERVER_VERSION = "1.0.0"

print(f"Server: {SERVER_NAME} v{SERVER_VERSION}")

---

## 2. Building Your First MCP Server

### 2.1 Minimal MCP Server

In [ ]:
from mcp.server import Server
from mcp.types import TextContent

# Create server instance
server = Server("hello-world-server")

# Define a simple tool
@server.tool()
async def greet(name: str) -> list[TextContent]:
    """Greet a user by name"""
    return [TextContent(
        type="text",
        text=f"Hello, {name}! Welcome to MCP."
    )]

print("✅ Server created with 'greet' tool")
print(f"Server name: {server.name}")

### 2.2 Running the Server (stdio mode)

**Note:** This is how you would run the server as a standalone script:

```python
# server.py
from mcp.server import Server

server = Server("my-server")

@server.tool()
async def my_tool(param: str) -> list[TextContent]:
    return [TextContent(type="text", text=f"Result: {param}")]

if __name__ == "__main__":
    # Run server with stdio transport
    server.run()
```

**Usage:**
```bash
python server.py
```

---

## 3. Implementing Tools

### 3.1 Simple Tool - Calculator

In [ ]:
from mcp.server import Server
from mcp.types import TextContent
from typing import Literal

calc_server = Server("calculator-server")

@calc_server.tool()
async def calculate(
    a: float,
    b: float,
    operation: Literal["add", "subtract", "multiply", "divide"]
) -> list[TextContent]:
    """Perform basic arithmetic operations"""
    
    if operation == "add":
        result = a + b
    elif operation == "subtract":
        result = a - b
    elif operation == "multiply":
        result = a * b
    elif operation == "divide":
        if b == 0:
            return [TextContent(type="text", text="Error: Division by zero")]
        result = a / b
    
    return [TextContent(
        type="text",
        text=f"{a} {operation} {b} = {result}"
    )]

# Test the tool (simulated)
import asyncio

async def test_calculator():
    result = await calculate(10, 5, "add")
    print(result[0].text)
    
    result = await calculate(10, 0, "divide")
    print(result[0].text)

await test_calculator()
print("\n✅ Calculator tool implemented successfully")

### 3.2 Tool with External API Call - Weather Service

In [ ]:
import httpx
from mcp.server import Server
from mcp.types import TextContent

weather_server = Server("weather-server")

@weather_server.tool()
async def get_weather(city: str, units: str = "metric") -> list[TextContent]:
    """
    Get current weather for a city
    
    Args:
        city: City name (e.g., "London", "New York")
        units: Temperature units ("metric" or "imperial")
    """
    
    # Using free OpenWeatherMap-like API (example)
    api_key = os.getenv("WEATHER_API_KEY", "demo_key")
    
    try:
        async with httpx.AsyncClient() as client:
            # Example URL (replace with actual API)
            url = f"https://api.openweathermap.org/data/2.5/weather"
            params = {
                "q": city,
                "appid": api_key,
                "units": units
            }
            
            # For demo purposes, return mock data
            # In production, uncomment:
            # response = await client.get(url, params=params)
            # response.raise_for_status()
            # data = response.json()
            
            # Mock response
            data = {
                "main": {"temp": 22.5, "humidity": 65},
                "weather": [{"description": "partly cloudy"}],
                "name": city
            }
            
            temp_unit = "°C" if units == "metric" else "°F"
            weather_text = (
                f"Weather in {data['name']}:\n"
                f"Temperature: {data['main']['temp']}{temp_unit}\n"
                f"Humidity: {data['main']['humidity']}%\n"
                f"Conditions: {data['weather'][0]['description']}"
            )
            
            return [TextContent(type="text", text=weather_text)]
            
    except Exception as e:
        return [TextContent(
            type="text",
            text=f"Error fetching weather: {str(e)}"
        )]

# Test the weather tool
async def test_weather():
    result = await get_weather("London")
    print(result[0].text)

await test_weather()
print("\n✅ Weather tool with API call implemented")

### 3.3 Tool with File Operations - File Manager

In [ ]:
from pathlib import Path
from mcp.server import Server
from mcp.types import TextContent

file_server = Server("file-manager-server")

@file_server.tool()
async def create_file(path: str, content: str) -> list[TextContent]:
    """
    Create a new file with content
    
    Args:
        path: File path (relative or absolute)
        content: Content to write to file
    """
    try:
        # Security: Ensure path is within allowed directory
        allowed_base = Path("/tmp/mcp-demo")
        allowed_base.mkdir(parents=True, exist_ok=True)
        
        file_path = (allowed_base / path).resolve()
        
        # Prevent path traversal
        if not str(file_path).startswith(str(allowed_base)):
            return [TextContent(
                type="text",
                text="Error: Path traversal detected. Access denied."
            )]
        
        # Create parent directories
        file_path.parent.mkdir(parents=True, exist_ok=True)
        
        # Write file
        file_path.write_text(content)
        
        return [TextContent(
            type="text",
            text=f"✅ File created successfully: {file_path}\nSize: {len(content)} bytes"
        )]
        
    except Exception as e:
        return [TextContent(
            type="text",
            text=f"Error creating file: {str(e)}"
        )]

@file_server.tool()
async def read_file(path: str) -> list[TextContent]:
    """
    Read content from a file
    
    Args:
        path: File path to read
    """
    try:
        allowed_base = Path("/tmp/mcp-demo")
        file_path = (allowed_base / path).resolve()
        
        if not str(file_path).startswith(str(allowed_base)):
            return [TextContent(
                type="text",
                text="Error: Path traversal detected. Access denied."
            )]
        
        if not file_path.exists():
            return [TextContent(
                type="text",
                text=f"Error: File not found: {path}"
            )]
        
        content = file_path.read_text()
        
        return [TextContent(
            type="text",
            text=f"Content of {path}:\n\n{content}"
        )]
        
    except Exception as e:
        return [TextContent(
            type="text",
            text=f"Error reading file: {str(e)}"
        )]

# Test file operations
async def test_file_operations():
    # Create a file
    result = await create_file("test.txt", "Hello from MCP!")
    print(result[0].text)
    
    # Read the file
    result = await read_file("test.txt")
    print("\n" + result[0].text)
    
    # Test path traversal protection
    result = await read_file("../../etc/passwd")
    print("\n" + result[0].text)

await test_file_operations()
print("\n✅ File operations tools implemented with security")

### 3.4 Tool with Error Handling & Retry Logic

In [ ]:
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type
import httpx
from mcp.server import Server
from mcp.types import TextContent

resilient_server = Server("resilient-server")

class APIError(Exception):
    """Custom exception for API errors"""
    pass

@retry(
    stop=stop_after_attempt(3),
    wait=wait_exponential(multiplier=1, min=1, max=10),
    retry=retry_if_exception_type(httpx.HTTPStatusError),
    reraise=True
)
async def fetch_with_retry(url: str) -> dict:
    """Fetch data with automatic retry on failures"""
    async with httpx.AsyncClient() as client:
        response = await client.get(url, timeout=10.0)
        response.raise_for_status()
        return response.json()

@resilient_server.tool()
async def fetch_data(url: str) -> list[TextContent]:
    """
    Fetch data from URL with retry logic
    
    Args:
        url: URL to fetch data from
    """
    try:
        # For demo, simulate with mock data
        # In production, use: data = await fetch_with_retry(url)
        
        data = {
            "status": "success",
            "message": "Data fetched successfully",
            "url": url
        }
        
        return [TextContent(
            type="text",
            text=f"✅ Fetched data from {url}:\n{data}"
        )]
        
    except httpx.HTTPStatusError as e:
        return [TextContent(
            type="text",
            text=f"❌ HTTP Error after retries: {e.response.status_code}"
        )]
    except httpx.TimeoutException:
        return [TextContent(
            type="text",
            text="❌ Request timed out after retries"
        )]
    except Exception as e:
        return [TextContent(
            type="text",
            text=f"❌ Unexpected error: {str(e)}"
        )]

# Test resilient tool
async def test_resilient_tool():
    result = await fetch_data("https://api.example.com/data")
    print(result[0].text)

await test_resilient_tool()
print("\n✅ Resilient tool with retry logic implemented")

---

## 4. Implementing Resources

### 4.1 Static Resource - Configuration

In [ ]:
from mcp.server import Server
from mcp.types import Resource, TextContent
import json

resource_server = Server("resource-demo-server")

@resource_server.list_resources()
async def list_resources() -> list[Resource]:
    """
    List all available resources
    """
    return [
        Resource(
            uri="resource://config/server-settings",
            name="Server Settings",
            description="Server configuration settings",
            mimeType="application/json"
        ),
        Resource(
            uri="resource://config/api-endpoints",
            name="API Endpoints",
            description="Available API endpoints",
            mimeType="application/json"
        )
    ]

@resource_server.read_resource()
async def read_resource(uri: str) -> str:
    """
    Read resource by URI
    """
    if uri == "resource://config/server-settings":
        config = {
            "server_name": "MCP Demo Server",
            "version": "1.0.0",
            "max_connections": 100,
            "timeout": 30,
            "features": ["tools", "resources", "prompts"]
        }
        return json.dumps(config, indent=2)
    
    elif uri == "resource://config/api-endpoints":
        endpoints = {
            "endpoints": [
                {"path": "/tools/list", "method": "GET"},
                {"path": "/tools/call", "method": "POST"},
                {"path": "/resources/list", "method": "GET"},
                {"path": "/resources/read", "method": "POST"}
            ]
        }
        return json.dumps(endpoints, indent=2)
    
    else:
        raise ValueError(f"Resource not found: {uri}")

# Test resources
async def test_resources():
    # List resources
    resources = await list_resources()
    print("Available Resources:")
    for r in resources:
        print(f"  - {r.name}: {r.uri}")
    
    # Read a resource
    print("\nReading server-settings resource:")
    content = await read_resource("resource://config/server-settings")
    print(content)

await test_resources()
print("\n✅ Static resources implemented")

### 4.2 Dynamic Resource - Database Query Results

In [ ]:
from mcp.server import Server
from mcp.types import Resource
import json
from datetime import datetime

db_resource_server = Server("database-resource-server")

# Mock database
MOCK_DATABASE = {
    "users": [
        {"id": 1, "name": "Alice", "email": "alice@example.com"},
        {"id": 2, "name": "Bob", "email": "bob@example.com"},
        {"id": 3, "name": "Charlie", "email": "charlie@example.com"}
    ],
    "orders": [
        {"id": 101, "user_id": 1, "amount": 99.99, "status": "completed"},
        {"id": 102, "user_id": 2, "amount": 149.99, "status": "pending"}
    ]
}

@db_resource_server.list_resources()
async def list_db_resources() -> list[Resource]:
    """
    List database tables as resources
    """
    resources = []
    for table_name in MOCK_DATABASE.keys():
        resources.append(Resource(
            uri=f"resource://database/{table_name}",
            name=f"{table_name.capitalize()} Table",
            description=f"Data from {table_name} table",
            mimeType="application/json"
        ))
    return resources

@db_resource_server.read_resource()
async def read_db_resource(uri: str) -> str:
    """
    Read database table data
    """
    # Extract table name from URI
    if uri.startswith("resource://database/"):
        table_name = uri.split("/")[-1]
        
        if table_name in MOCK_DATABASE:
            data = {
                "table": table_name,
                "rows": MOCK_DATABASE[table_name],
                "count": len(MOCK_DATABASE[table_name]),
                "timestamp": datetime.now().isoformat()
            }
            return json.dumps(data, indent=2)
        else:
            raise ValueError(f"Table not found: {table_name}")
    
    raise ValueError(f"Invalid resource URI: {uri}")

# Test dynamic resources
async def test_db_resources():
    resources = await list_db_resources()
    print("Database Resources:")
    for r in resources:
        print(f"  - {r.name}: {r.uri}")
    
    print("\nReading users table:")
    content = await read_db_resource("resource://database/users")
    print(content)

await test_db_resources()
print("\n✅ Dynamic database resources implemented")

---

## 5. Implementing Prompts

### 5.1 Simple Prompt Template

In [ ]:
from mcp.server import Server
from mcp.types import Prompt, PromptArgument, PromptMessage

prompt_server = Server("prompt-demo-server")

@prompt_server.list_prompts()
async def list_prompts() -> list[Prompt]:
    """
    List available prompt templates
    """
    return [
        Prompt(
            name="code_review",
            description="Review code for quality and security",
            arguments=[
                PromptArgument(
                    name="language",
                    description="Programming language",
                    required=True
                ),
                PromptArgument(
                    name="code",
                    description="Code to review",
                    required=True
                ),
                PromptArgument(
                    name="focus",
                    description="Focus areas (security, performance, style)",
                    required=False
                )
            ]
        ),
        Prompt(
            name="summarize_document",
            description="Summarize a document",
            arguments=[
                PromptArgument(
                    name="document",
                    description="Document text to summarize",
                    required=True
                ),
                PromptArgument(
                    name="length",
                    description="Summary length (short, medium, long)",
                    required=False
                )
            ]
        )
    ]

@prompt_server.get_prompt()
async def get_prompt(name: str, arguments: dict) -> list[PromptMessage]:
    """
    Get filled prompt template
    """
    if name == "code_review":
        language = arguments.get("language")
        code = arguments.get("code")
        focus = arguments.get("focus", "all aspects")
        
        prompt_text = f"""You are an expert code reviewer. Please review the following {language} code.

Focus on: {focus}

Code to review:
```{language}
{code}
```

Provide a detailed review covering:
1. Code quality and best practices
2. Security vulnerabilities
3. Performance issues
4. Maintainability concerns

Format your response with specific line numbers and actionable suggestions."""
        
        return [PromptMessage(role="user", content=prompt_text)]
    
    elif name == "summarize_document":
        document = arguments.get("document")
        length = arguments.get("length", "medium")
        
        length_guidance = {
            "short": "2-3 sentences",
            "medium": "1-2 paragraphs",
            "long": "3-4 paragraphs with key details"
        }
        
        prompt_text = f"""Please provide a {length} summary of the following document.

Document:
{document}

Summary length: {length_guidance.get(length, length)}

Include:
- Main points
- Key takeaways
- Important details"""
        
        return [PromptMessage(role="user", content=prompt_text)]
    
    raise ValueError(f"Unknown prompt: {name}")

# Test prompts
async def test_prompts():
    prompts = await list_prompts()
    print("Available Prompts:")
    for p in prompts:
        print(f"  - {p.name}: {p.description}")
    
    print("\nGetting code_review prompt:")
    messages = await get_prompt(
        "code_review",
        {
            "language": "python",
            "code": "def add(a, b):\n    return a + b",
            "focus": "security and performance"
        }
    )
    print(messages[0].content)

await test_prompts()
print("\n✅ Prompt templates implemented")

---

## 6. Building an MCP Client

### 6.1 Simple MCP Client

In [ ]:
from mcp.client import ClientSession
from mcp.client.stdio import stdio_client
import asyncio

async def simple_mcp_client():
    """
    Example of MCP client connecting to server
    """
    # Note: This is a conceptual example
    # In practice, you'd connect to an actual running server
    
    print("MCP Client Example:")
    print("1. Initialize client session")
    print("2. Connect to server (stdio or HTTP)")
    print("3. Discover capabilities (tools/resources/prompts)")
    print("4. Call tools or read resources")
    print("5. Close connection")
    
    # Pseudo-code for actual client implementation:
    """
    async with stdio_client(
        command="python",
        args=["server.py"]
    ) as (read, write):
        async with ClientSession(read, write) as session:
            # Initialize
            await session.initialize()
            
            # List tools
            tools = await session.list_tools()
            print(f"Available tools: {[t.name for t in tools]}")
            
            # Call a tool
            result = await session.call_tool(
                "greet",
                arguments={"name": "World"}
            )
            print(f"Tool result: {result}")
    """
    
    print("\n✅ Client implementation pattern demonstrated")

await simple_mcp_client()

---

## 7. Real-World Examples

### 7.1 GitHub Integration Server

In [ ]:
from mcp.server import Server
from mcp.types import TextContent, Resource
import httpx
import json

github_server = Server("github-integration")

# GitHub API base URL
GITHUB_API = "https://api.github.com"

@github_server.tool()
async def create_issue(
    repo: str,
    title: str,
    body: str,
    labels: list[str] = None
) -> list[TextContent]:
    """
    Create a GitHub issue
    
    Args:
        repo: Repository in format "owner/repo"
        title: Issue title
        body: Issue description
        labels: List of label names
    """
    # Mock implementation (replace with actual API call in production)
    token = os.getenv("GITHUB_TOKEN", "demo_token")
    
    issue_data = {
        "title": title,
        "body": body,
        "labels": labels or []
    }
    
    # Mock response
    issue_number = 123
    issue_url = f"https://github.com/{repo}/issues/{issue_number}"
    
    return [TextContent(
        type="text",
        text=f"✅ Issue created successfully!\n\nIssue #{issue_number}: {title}\nURL: {issue_url}"
    )]

@github_server.tool()
async def search_code(
    query: str,
    repo: str = None,
    language: str = None
) -> list[TextContent]:
    """
    Search code on GitHub
    
    Args:
        query: Search query
        repo: Optional repository filter (owner/repo)
        language: Optional language filter
    """
    # Build search query
    full_query = query
    if repo:
        full_query += f" repo:{repo}"
    if language:
        full_query += f" language:{language}"
    
    # Mock search results
    results = [
        {
            "file": "src/main.py",
            "repo": repo or "example/repo",
            "line": 42,
            "snippet": "def process_data(data):\n    return data"
        }
    ]
    
    results_text = f"Search results for '{query}':\n\n"
    for i, r in enumerate(results, 1):
        results_text += f"{i}. {r['repo']}/{r['file']}:{r['line']}\n{r['snippet']}\n\n"
    
    return [TextContent(type="text", text=results_text)]

@github_server.list_resources()
async def list_github_resources() -> list[Resource]:
    """
    List GitHub resources (repos, PRs, issues)
    """
    return [
        Resource(
            uri="resource://github/repos/starred",
            name="Starred Repositories",
            description="User's starred repositories",
            mimeType="application/json"
        ),
        Resource(
            uri="resource://github/pulls/open",
            name="Open Pull Requests",
            description="User's open pull requests",
            mimeType="application/json"
        )
    ]

# Test GitHub integration
async def test_github_server():
    # Create issue
    result = await create_issue(
        "anthropics/claude-code",
        "Add MCP support",
        "We should add Model Context Protocol support",
        ["enhancement", "mcp"]
    )
    print(result[0].text)
    
    # Search code
    print("\n" + "="*50)
    result = await search_code("async def", language="python")
    print(result[0].text)

await test_github_server()
print("✅ GitHub integration server implemented")

### 7.2 Database Query Server

In [ ]:
from mcp.server import Server
from mcp.types import TextContent
import json

db_server = Server("database-query-server")

# Mock database for demonstration
MOCK_DB = {
    "users": [
        {"id": 1, "name": "Alice", "role": "admin"},
        {"id": 2, "name": "Bob", "role": "user"},
        {"id": 3, "name": "Charlie", "role": "user"}
    ],
    "orders": [
        {"id": 101, "user_id": 1, "total": 99.99},
        {"id": 102, "user_id": 2, "total": 149.99},
        {"id": 103, "user_id": 1, "total": 79.99}
    ]
}

@db_server.tool()
async def query_table(
    table: str,
    filter_field: str = None,
    filter_value: str = None
) -> list[TextContent]:
    """
    Query database table with optional filter
    
    Args:
        table: Table name to query
        filter_field: Optional field to filter by
        filter_value: Optional value for filter
    """
    if table not in MOCK_DB:
        return [TextContent(
            type="text",
            text=f"Error: Table '{table}' not found"
        )]
    
    results = MOCK_DB[table]
    
    # Apply filter if provided
    if filter_field and filter_value:
        results = [
            row for row in results
            if str(row.get(filter_field)) == str(filter_value)
        ]
    
    result_text = f"Query results from '{table}' table:\n\n"
    result_text += json.dumps(results, indent=2)
    result_text += f"\n\nTotal rows: {len(results)}"
    
    return [TextContent(type="text", text=result_text)]

@db_server.tool()
async def aggregate_data(
    table: str,
    operation: str,
    field: str
) -> list[TextContent]:
    """
    Perform aggregation on table data
    
    Args:
        table: Table name
        operation: Aggregation operation (sum, avg, count, min, max)
        field: Field to aggregate
    """
    if table not in MOCK_DB:
        return [TextContent(
            type="text",
            text=f"Error: Table '{table}' not found"
        )]
    
    data = MOCK_DB[table]
    values = [row.get(field) for row in data if field in row]
    
    if not values:
        return [TextContent(
            type="text",
            text=f"Error: Field '{field}' not found or has no values"
        )]
    
    if operation == "sum":
        result = sum(values)
    elif operation == "avg":
        result = sum(values) / len(values)
    elif operation == "count":
        result = len(values)
    elif operation == "min":
        result = min(values)
    elif operation == "max":
        result = max(values)
    else:
        return [TextContent(
            type="text",
            text=f"Error: Unknown operation '{operation}'"
        )]
    
    return [TextContent(
        type="text",
        text=f"{operation.upper()}({field}) from {table}: {result}"
    )]

# Test database server
async def test_db_server():
    # Query all users
    result = await query_table("users")
    print(result[0].text)
    
    # Query with filter
    print("\n" + "="*50)
    result = await query_table("users", "role", "user")
    print(result[0].text)
    
    # Aggregate
    print("\n" + "="*50)
    result = await aggregate_data("orders", "sum", "total")
    print(result[0].text)

await test_db_server()
print("\n✅ Database query server implemented")

---

## 8. Testing & Debugging

### 8.1 Unit Testing MCP Tools

In [ ]:
import pytest
from mcp.server import Server
from mcp.types import TextContent

# Example test suite for MCP server

test_server = Server("test-server")

@test_server.tool()
async def add_numbers(a: int, b: int) -> list[TextContent]:
    """Add two numbers"""
    return [TextContent(
        type="text",
        text=str(a + b)
    )]

# Test cases
async def test_add_numbers_positive():
    """Test adding positive numbers"""
    result = await add_numbers(5, 3)
    assert result[0].text == "8"
    print("✅ test_add_numbers_positive passed")

async def test_add_numbers_negative():
    """Test adding negative numbers"""
    result = await add_numbers(-5, 3)
    assert result[0].text == "-2"
    print("✅ test_add_numbers_negative passed")

async def test_add_numbers_zero():
    """Test adding with zero"""
    result = await add_numbers(0, 0)
    assert result[0].text == "0"
    print("✅ test_add_numbers_zero passed")

# Run tests
async def run_tests():
    await test_add_numbers_positive()
    await test_add_numbers_negative()
    await test_add_numbers_zero()
    print("\n✅ All tests passed!")

await run_tests()

### 8.2 Debugging Tips

In [ ]:
import logging
from mcp.server import Server
from mcp.types import TextContent

# Configure logging for debugging
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

debug_server = Server("debug-server")

@debug_server.tool()
async def debuggable_tool(input_data: str) -> list[TextContent]:
    """
    Tool with debugging instrumentation
    """
    logger.debug(f"Tool called with input: {input_data}")
    
    try:
        # Process data
        result = input_data.upper()
        logger.info(f"Processing successful: {result}")
        
        return [TextContent(type="text", text=result)]
        
    except Exception as e:
        logger.error(f"Error in tool execution: {e}", exc_info=True)
        return [TextContent(
            type="text",
            text=f"Error: {str(e)}"
        )]

# Test with logging
result = await debuggable_tool("hello world")
print(f"\nResult: {result[0].text}")
print("\n✅ Debugging instrumentation demonstrated")

---

## 9. Deployment Patterns

### 9.1 Claude Desktop Configuration

Add to `~/Library/Application Support/Claude/claude_desktop_config.json` (macOS):

```json
{
  "mcpServers": {
    "my-custom-server": {
      "command": "python",
      "args": ["/path/to/server.py"],
      "env": {
        "API_KEY": "your-api-key"
      }
    }
  }
}
```

### 9.2 Standalone Server Script

In [ ]:
# Save this as server.py

PRODUCTION_SERVER_TEMPLATE = '''
#!/usr/bin/env python3
"""
Production MCP Server
"""
from mcp.server import Server
from mcp.types import TextContent
import logging
import os

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('/var/log/mcp-server.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# Create server
server = Server("production-server")

@server.tool()
async def example_tool(param: str) -> list[TextContent]:
    """Example production tool"""
    logger.info(f"Tool called with: {param}")
    return [TextContent(type="text", text=f"Processed: {param}")]

if __name__ == "__main__":
    logger.info("Starting MCP server...")
    try:
        server.run()
    except KeyboardInterrupt:
        logger.info("Server stopped by user")
    except Exception as e:
        logger.error(f"Server error: {e}", exc_info=True)
'''

print("Production server template:")
print(PRODUCTION_SERVER_TEMPLATE)
print("\n✅ Save this as server.py to deploy")

---

## 10. Complete Project Examples

### 10.1 All-in-One Demo Server

This combines tools, resources, and prompts in a single production-ready server:

In [ ]:
from mcp.server import Server
from mcp.types import Tool, Resource, Prompt, TextContent, PromptMessage, PromptArgument
import json
from datetime import datetime

# Create comprehensive demo server
demo_server = Server("comprehensive-demo-server")

# ========== TOOLS ==========

@demo_server.tool()
async def get_system_info() -> list[TextContent]:
    """Get system information"""
    import platform
    info = {
        "platform": platform.system(),
        "python_version": platform.python_version(),
        "timestamp": datetime.now().isoformat()
    }
    return [TextContent(
        type="text",
        text=json.dumps(info, indent=2)
    )]

@demo_server.tool()
async def process_text(
    text: str,
    operation: str
) -> list[TextContent]:
    """Process text with various operations"""
    operations = {
        "uppercase": text.upper(),
        "lowercase": text.lower(),
        "reverse": text[::-1],
        "length": f"Length: {len(text)}"
    }
    
    result = operations.get(operation, f"Unknown operation: {operation}")
    return [TextContent(type="text", text=str(result))]

# ========== RESOURCES ==========

@demo_server.list_resources()
async def list_demo_resources() -> list[Resource]:
    return [
        Resource(
            uri="resource://demo/config",
            name="Server Configuration",
            description="Current server configuration",
            mimeType="application/json"
        ),
        Resource(
            uri="resource://demo/stats",
            name="Server Statistics",
            description="Current server statistics",
            mimeType="application/json"
        )
    ]

@demo_server.read_resource()
async def read_demo_resource(uri: str) -> str:
    if uri == "resource://demo/config":
        config = {
            "name": "Comprehensive Demo Server",
            "version": "1.0.0",
            "features": ["tools", "resources", "prompts"],
            "max_connections": 100
        }
        return json.dumps(config, indent=2)
    
    elif uri == "resource://demo/stats":
        stats = {
            "uptime_seconds": 3600,
            "requests_handled": 1234,
            "active_connections": 5
        }
        return json.dumps(stats, indent=2)
    
    raise ValueError(f"Unknown resource: {uri}")

# ========== PROMPTS ==========

@demo_server.list_prompts()
async def list_demo_prompts() -> list[Prompt]:
    return [
        Prompt(
            name="analyze_data",
            description="Analyze data and provide insights",
            arguments=[
                PromptArgument(
                    name="data",
                    description="Data to analyze",
                    required=True
                )
            ]
        )
    ]

@demo_server.get_prompt()
async def get_demo_prompt(name: str, arguments: dict) -> list[PromptMessage]:
    if name == "analyze_data":
        data = arguments.get("data")
        prompt = f"""Analyze the following data and provide insights:

{data}

Include:
1. Key patterns
2. Anomalies
3. Recommendations"""
        return [PromptMessage(role="user", content=prompt)]
    
    raise ValueError(f"Unknown prompt: {name}")

# ========== TEST ALL FEATURES ==========

async def test_comprehensive_server():
    print("=" * 60)
    print("COMPREHENSIVE DEMO SERVER TEST")
    print("=" * 60)
    
    # Test tools
    print("\n📦 TESTING TOOLS:")
    result = await get_system_info()
    print(f"\n1. get_system_info:\n{result[0].text}")
    
    result = await process_text("Hello MCP", "uppercase")
    print(f"\n2. process_text:\n{result[0].text}")
    
    # Test resources
    print("\n\n📚 TESTING RESOURCES:")
    resources = await list_demo_resources()
    print(f"\nAvailable resources: {len(resources)}")
    for r in resources:
        print(f"  - {r.name}")
    
    content = await read_demo_resource("resource://demo/config")
    print(f"\nConfig resource:\n{content}")
    
    # Test prompts
    print("\n\n💬 TESTING PROMPTS:")
    prompts = await list_demo_prompts()
    print(f"\nAvailable prompts: {len(prompts)}")
    for p in prompts:
        print(f"  - {p.name}: {p.description}")
    
    messages = await get_demo_prompt(
        "analyze_data",
        {"data": "[1, 2, 3, 4, 5]"}
    )
    print(f"\nGenerated prompt:\n{messages[0].content}")
    
    print("\n" + "=" * 60)
    print("✅ ALL TESTS PASSED!")
    print("=" * 60)

await test_comprehensive_server()

---

## Summary

### What We've Covered:

1. ✅ **Setup & Installation** - MCP SDK and dependencies
2. ✅ **Building Servers** - From minimal to comprehensive
3. ✅ **Implementing Tools** - Calculator, weather API, file operations, error handling
4. ✅ **Implementing Resources** - Static and dynamic data sources
5. ✅ **Implementing Prompts** - Reusable templates with parameters
6. ✅ **Building Clients** - Connecting to MCP servers
7. ✅ **Real-World Examples** - GitHub integration, database queries
8. ✅ **Testing & Debugging** - Unit tests and logging
9. ✅ **Deployment** - Claude Desktop config, standalone scripts
10. ✅ **Complete Projects** - All-in-one demo server

### Next Steps:

1. **Practice:** Modify the examples to create your own tools
2. **Deploy:** Configure Claude Desktop to use your server
3. **Extend:** Add more sophisticated tools and resources
4. **Production:** Implement security, monitoring, and error handling
5. **Share:** Publish your MCP server for others to use

### Resources:

- [MCP Documentation](https://modelcontextprotocol.io)
- [MCP Python SDK](https://github.com/modelcontextprotocol/python-sdk)
- [MCP Servers Repository](https://github.com/modelcontextprotocol/servers)
- [Anthropic MCP Guide](https://docs.anthropic.com/mcp)

---

**Happy building with MCP! 🚀**

*Created: January 2025*  
*Version: 1.0*  
*For interview preparation and learning*